In [ ]:
import pandas as pd

len(pd.read_json('/home/al/projects/idx-bei/data/companyDetailsByKodeEmiten.json').columns)

In [ ]:
with open("/home/al/projects/idx-bei/data/companyDetailsByKodeEmiten.json", "r", encoding="utf-8") as f:
    stocks = json.load(f)  # Make sure this returns a list of dicts


In [ ]:
print(type(stocks))         # should be <class 'list'>
print(type(stocks[0]))      # should be <class 'dict'>


In [ ]:
import json
from neo4j import GraphDatabase

# Neo4j connection config
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"

# Load JSON data
with open("/home/al/projects/idx-bei/data/companyDetailsByKodeEmiten.json", "r", encoding="utf-8") as f:
    stocks = json.load(f)

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def ingest_stock(tx, stock):
    # Create Company node
    tx.run("""
        MERGE (c:Company {kode: $kode})
        SET c.name = $name, c.industry = $industry, c.sector = $sector, c.website = $website
    """, kode=stock['Profiles'][0]["KodeEmiten"], name=stock['Profiles'][0]["NamaEmiten"], industry=stock['Profiles'][0]["Industri"], sector=stock['Profiles'][0]["Sektor"], website=stock['Profiles'][0]["Website"])

    # Directors
    for d in stock.get("Direksi", []):
        tx.run("""
            MERGE (d:Insider {name: $name})
            SET d.jabatan = $jabatan, d.afiliasi = $afiliasi
            WITH d
            MATCH (c:Company {kode: $kode})
            MERGE (c)-[:HAS_DIRECTOR]->(d)
        """, name=d["Nama"], jabatan=d["Jabatan"], afiliasi=d.get("Afiliasi", False), kode=stock['Profiles'][0]["KodeEmiten"])

    # Commissioners
    for k in stock.get("Komisaris", []):
        tx.run("""
            MERGE (k:Insider {name: $name})
            SET k.jabatan = $jabatan, k.independen = $independen
            WITH k
            MATCH (c:Company {kode: $kode})
            MERGE (c)-[:HAS_COMMISSIONER]->(k)
        """, name=k["Nama"], jabatan=k["Jabatan"], independen=k.get("Independen", False), kode=stock['Profiles'][0]["KodeEmiten"])

    # Shareholders
    for s in stock.get("PemegangSaham", []):
        tx.run("""
            MERGE (s:Insider {name: $name})
            SET s.kategori = $kategori
            WITH s
            MATCH (c:Company {kode: $kode})
            MERGE (s)-[:OWNS {persentase: $persentase, jumlah: $jumlah}]->(c)
        """, name=s["Nama"], kategori=s.get("Kategori"), persentase=s.get("Persentase"), jumlah=s.get("Jumlah"), kode=stock['Profiles'][0]["KodeEmiten"])

# Ingest all stocks
with driver.session() as session:
    for ticker, stock_data in stocks.items():
        # print(ticker, stock_data['Profiles'][0][0]["KodeEmiten"])
        session.execute_write(ingest_stock, stock_data) 

print("Ingestion complete.")
driver.close()
